In [1]:
#数据预处理
%matplotlib inline
import numpy as np
#import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import missingno as msno

#不加下面这一步进程很容易中断
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ["OMP_NUM_THREADS"] = '9' 

#---------------------------------------------------
import sklearn
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay
#加载模型：
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

#---------------------------
from sklearn.model_selection import GridSearchCV

In [2]:
#一些函数
# get_train_test
# get_dat->train_model
# model

#获得分折好的数据集
def get_data(data,name):
    train_data = data #train[0] / test[0]
    target_name = name #"y4"
    #读
    df= train_data
    features=df.columns.tolist()
    features.remove( target_name )
    y=df[ target_name ]
    X=df[features].values
    return X,y

In [3]:
#读取最开始的数据
fpath="./lasso/5_train_liux_subx.xlsx"
df=pd.read_excel(fpath)

In [4]:
X,y = get_data(df,"随访>=3")
"""
import xgboost as xgb
para = {'max_depth': 12, 'min_child_weight': 0.7, 'n_estimators': 20, 'scale_pos_weight': 2.4}
model = xgb.XGBClassifier(**para)
"""
from sklearn.linear_model import LogisticRegression
para = {'C': 5, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}
model = LogisticRegression(**para)
model.fit(X, y)


# 预测概率
y_probabilities_1 = model.predict_proba(X)


#读取最开始的数据
fpath="./lasso/6_test_liux_subx.xlsx"
df=pd.read_excel(fpath)
X,y = get_data(df,"随访>=3")
y_probabilities_2 = model.predict_proba(X)

In [5]:
y_probabilities_1[:,1]
y_probabilities_2[:,1]

array([0.32004706, 0.52619007, 0.29980969, 0.32210907, 0.44477471,
       0.10177257, 0.38664522, 0.51060238, 0.44098417, 0.54479378,
       0.25767874, 0.22414332, 0.33236832, 0.41759453, 0.48042696,
       0.37825332, 0.18002674, 0.50231153, 0.39277882, 0.42567555,
       0.31451999, 0.48294811, 0.40638905, 0.1921348 , 0.37164979,
       0.16552808, 0.32241533, 0.32036214, 0.41932873, 0.36164526,
       0.35714023, 0.44523844, 0.41634317, 0.33700572, 0.57290033,
       0.52988979, 0.35674968, 0.45049194, 0.34579836, 0.51276262,
       0.45177836, 0.34632765, 0.30347424, 0.14400413, 0.42026837,
       0.45723901, 0.24740169, 0.33536993, 0.52098585, 0.3206726 ,
       0.24280396, 0.36404115, 0.40131937, 0.10551361, 0.39359386,
       0.42718359, 0.57553399, 0.24947928])

In [6]:
np.concatenate((y_probabilities_1[:,0], y_probabilities_2[:,0]))

array([0.14343579, 0.18822419, 0.42627226, 0.75456833, 0.45219388,
       0.52213178, 0.68674172, 0.15554143, 0.7866223 , 0.46718563,
       0.66236046, 0.443514  , 0.52208676, 0.64613586, 0.44513925,
       0.8332276 , 0.50239539, 0.46277172, 0.66622309, 0.64744147,
       0.145252  , 0.43099026, 0.57298011, 0.51474898, 0.65176723,
       0.36949836, 0.7237595 , 0.55490606, 0.43483818, 0.20353146,
       0.65431932, 0.26950819, 0.50895358, 0.33230487, 0.29507819,
       0.54377151, 0.17045208, 0.39613167, 0.14913889, 0.45982691,
       0.59312913, 0.41015124, 0.2999069 , 0.43085353, 0.39487938,
       0.18651469, 0.53688987, 0.63395501, 0.47747915, 0.58270722,
       0.83664778, 0.73477565, 0.45840139, 0.53123086, 0.28659047,
       0.83813785, 0.34143749, 0.43867476, 0.56111692, 0.33691042,
       0.24693023, 0.58016639, 0.89477932, 0.42371883, 0.53243081,
       0.57402175, 0.37687585, 0.46086627, 0.37381739, 0.305787  ,
       0.30635437, 0.83314687, 0.65042306, 0.4881548 , 0.83461

In [7]:
score = np.concatenate((y_probabilities_1[:,0], y_probabilities_2[:,0]))

In [8]:
score = pd.DataFrame({'score': score})

In [10]:
fpath_out="./lasso/7_score.xlsx"
score.to_excel(fpath_out,index=False)# index=False 避免为索引添加一列特征   #utf_8_sig   utf-8

In [11]:
fpath="./data/norm_3_train_med.xlsx"
df1=pd.read_excel(fpath)
df1 = df1.fillna(0)

fpath="./data/norm_4_test_med.xlsx"
df2=pd.read_excel(fpath)
df2 = df2.fillna(0)

df = pd.concat([df1,df2],axis=0)

In [13]:
grouped = df.groupby("随访>=3")
#grouped = df.groupby("随访>入院")

# 获取两个组的数据
group_a = grouped.get_group(1)
group_b = grouped.get_group(0)

In [14]:
#进行秩和检验（不需要样本长度一致），进行第一步筛选--------------------------------------

#https://zhuanlan.zhihu.com/p/555119816
#from scipy.stats import wilcoxon #需要样本长度一致
from scipy.stats import mannwhitneyu

alpha = 0.05

test_name = []
test_p = []
for i in group_a.columns.to_list()[:-1]:
    tp1 = group_a[i].tolist()
    tp2 = group_b[i].tolist()
    statistic, p_value = mannwhitneyu(tp1, tp2, alternative='two-sided')#wilcoxon(tp1, tp2)
    if i == "入路方向（2or3）":
        print(p_value)
    if p_value < alpha:
        #print("拒绝零假设，两个样本的中位数存在显著差异")
        #print("{0} - {1:.5f}".format( i , p_value))
        test_name.append(i)
        test_p.append(p_value)
    else:
        #print("未拒绝零假设，两个样本的中位数无显著差异")
        pass
print("------秩和检验筛选特征个数:",len(test_name),"------")
"""
if len(test_name)<=30:
    for i in range(len(test_name)):
        print("{0} - {1:.5f}".format( test_name[i] , test_p[i]))
else:
    top_30_indices = np.argsort(test_p)[:30]
    test_name = [test_name[i] for i in top_30_indices]
    test_p = [test_p[i] for i in top_30_indices]
    for i in range(len(test_name)):
        print("{0} - {1:.5f}".format( test_name[i] , test_p[i]))
"""
#如果只取前30个就只剩下sub2了  实际上181中1 2 3都有
for i in range(len(test_name)):
    print("{0} - {1:.5f}".format( test_name[i] , test_p[i]))

------秩和检验筛选特征个数: 8 ------
年龄 - 0.01619
既往治疗 - 0.01943
入路方向（2） - 0.00399
累计症状性出血次数 - 0.00100
出血时间 （3） - 0.03062
入院时MRS - 0.00002
是否神经功能障碍 - 0.01781
随访>3 - 0.00000


In [15]:
test_name.append("随访>=3")
test_name.remove("随访>3")

In [16]:
df = df[test_name]

In [17]:
df1 = df.head(160)
df2 = df.tail(58)

In [18]:
fpath_out="./lasso/8_train_score_med.xlsx"
df1.to_excel(fpath_out,index=False)# index=False 避免为索引添加一列特征   #utf_8_sig   utf-8

fpath_out="./lasso/9_test_score_med.xlsx"
df2.to_excel(fpath_out,index=False)# index=False 避免为索引添加一列特征   #utf_8_sig   utf-8